<a id="tableofcontents"></a>
# Table of Contents
- [Connect to synapse](#synapseconnect)
- [Subchallenge 1 data](#s1data)
- [Comparing Missing vs. Non-missing values](#missingornot)
- [Additional Analytics](#morecounts)

<a id="synapseconnect"></a>
## Connect to Synapse

In [1]:
import synapseclient
import synapseutils
import os
import configparser

config_path = os.path.join(os.environ['HOME'], '.synapseConfig')
cfg = configparser.ConfigParser()
cfg.read(config_path)
syn_usr = cfg.get("authentication", "username")
syn_pwd = cfg.get("authentication", "password")
syn = synapseclient.login(email=syn_usr, password=syn_pwd, rememberMe=True)

Welcome, Joy Rimchala!



<a id="s1data"></a>
# Subchallenge 1 data
Note we need to accept term of use on Synapse before the data can be downloaded.

back to [table of contents](#tableofcontents)

In [2]:
import numpy as np
import pandas as pd
from collections import OrderedDict
sub1_synapse_id = 'syn10139511'
if 'files' not in locals():
    files = synapseutils.syncFromSynapse(syn, sub1_synapse_id)

data_dict = OrderedDict()
for fl in files:
    data_dict[fl.name] = pd.read_csv(fl.path, sep='\t')


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!

In [ ]:
ground_truth['Gene_ID'].values

In [3]:
# For each Protein, what's the distribution of missing vs. non-missing values compared to truth
# Collect missing data stats across samples
missing = OrderedDict()
not_missing = OrderedDict()
ground_truth = data_dict['data_true.txt']
sample_cols = ground_truth.set_index('Gene_ID').columns
gene_list = ground_truth['Gene_ID'].values

for g in gene_list: 
    gtruth = ground_truth[ground_truth['Gene_ID']==g][sample_cols].values[0]
    for ds, obs in [(k, v) for (k, v) in data_dict.items() if not k.endswith('_true.txt')]:
        gobs = obs[obs['Gene_ID']==g][sample_cols].values[0]
        missing[ds, g] = gtruth[pd.isnull(gobs)]
        not_missing[ds, g] = gtruth[pd.notnull(gobs)]

        if g.endswith("500") & ds.endswith("9.txt"): 
            print("obs {}: gene: {}".format(ds, g))

obs data_obs_9.txt: gene: Protein_500
obs data_obs_9.txt: gene: Protein_1500
obs data_obs_9.txt: gene: Protein_2500
obs data_obs_9.txt: gene: Protein_3500
obs data_obs_9.txt: gene: Protein_4500
obs data_obs_9.txt: gene: Protein_5500
obs data_obs_9.txt: gene: Protein_6500
obs data_obs_9.txt: gene: Protein_7500


<a id="missingornot"></a>
## Comparing missing vs. non-missing values distributions
- [Aggregating values across samples by proteins](#byprotein)
- [Aggregating values across proteins by samples](#bysample)

back to [table of contents](#tableofcontents)

In [ ]:
# Utility functions
def _dim_dict():
    return {'sample':0, 'protein':1}    
    
def missing_categories_slice(missing, not_missing, _id, dim):
    # keys in missing dict is a tuple (sample_id, protein_id)
    all_samples_missing = []
    all_samples_not_missing = []
    for a in filter(lambda x: x[_dim_dict()[dim]] == _id, missing.keys()): 
        all_samples_missing.extend(missing[a])
        all_samples_not_missing.extend(not_missing[a])

    return all_samples_missing, all_samples_not_missing


def plot_histogram_missing_slice(all_samples_missing, all_samples_not_missing, _id, bin_missing=50, bin_not_missing=50):
    % pylab inline
    import seaborn as sns
    sns.set_style('white')
    sns.set_palette("hls", 4)

    fig, ax = plt.subplots(figsize=(5,4))
    alpha = 0.70
    ftsize = 20
    ax.hist(all_samples_not_missing, alpha=alpha, bins=bin_not_missing)
    ax.hist(all_samples_missing, alpha=alpha, bins=bin_missing)
    ax.grid('on')
    ax.set_title(_id, fontsize=ftsize)
    ax.set_xlabel('Protein level', fontsize=round(0.8*ftsize))
    ax.set_ylabel('number of samples', fontsize=round(0.8*ftsize))
    ax.legend(['not_missing', 'missing'], fontsize=round(0.7*ftsize))
    plt.show()

<a id="byprotein"></a>
### Aggregating values across samples by proteins

In [ ]:
protein_ids = unique(list(map(lambda x: x[_dim_dict()['protein']], missing.keys())))
num_samples = 5
random_proteins = protein_ids[np.random.randint(low=0, high=len(protein_ids), size=num_samples)]

for _id in random_proteins:
    all_samples_missing, all_samples_not_missing = missing_categories_slice(missing, not_missing, _id=_id, dim='protein')
    plot_histogram_missing_slice(all_samples_missing, all_samples_not_missing, _id, bin_missing=50, bin_not_missing=20)
    plt.show()

<a id="bysample"></a>
### Aggregating values across proteins by samples

In [ ]:
dim='sample'
_ids = unique(list(map(lambda x: x[_dim_dict()[dim]], missing.keys())))
num_samples = 5
randoms = _ids[np.random.randint(low=0, high=len(_ids), size=num_samples)]

for _id in randoms:
    all_samples_missing, all_samples_not_missing = missing_categories_slice(missing, not_missing, _id=_id, dim=dim)
    plot_histogram_missing_slice(all_samples_missing, all_samples_not_missing, _id, bin_missing=50, bin_not_missing=100)
    plt.show()

<a id="morecounts"></a>
## Additional Analytics
- Total number of missing values
- Total number of observations
- Missing value ratio
- Number of missing proteins per samples
- Proportions of zeros (NMAR) vs non-zeros (MAR) per samples

back to [table of contents](#tableofcontents)

In [ ]:
sample_cols = list(filter(lambda x: x.startswith('Sample'), data_dict['data_true.txt'].columns))
thres = 1
num_samples = 80
missing_counts_by_sample_by_protein = [
    (
        k[0], # observation id
        k[1], # protein id
        len(missing[k]), # total missing
        len([v for v in missing[k] if v >= thres]), # MAR
        len([v for v in missing[k] if v < thres])  # NMAR
    )
    for k in missing.keys()
]
missing_category = pd.DataFrame(missing_counts_by_sample_by_protein)
missing_category.columns = ['obs_id', 'Gene_ID', 'total_missing', 'MAR', 'NMAR']
missing_category['fraction_missing_total'] = missing_category['total_missing'].astype(float)/num_samples
missing_category['fraction_NMAR'] = missing_category['NMAR'].astype(float)/missing_category['total_missing']

not_missing_counts_by_sample_by_protein = [
    (
        k[0], # observation id
        k[1], # protein id
        len(not_missing[k]) # total not missing
    )
    for k in not_missing.keys()
]
not_missing_category = pd.DataFrame(not_missing_counts_by_sample_by_protein)
not_missing_category.columns = ['obs_id', 'Gene_ID', 'total_not_missing']
missing_category.to_csv('../data/missing_stats.csv', index=False)
not_missing_category.to_csv('../data/not_missing_stats.csv', index=False)

In [ ]:
total_number_of_missing_values = missing_category['total_missing'].sum()
num_samples = 80
total_observations = len(missing_category)*num_samples
print("Total Missing Values : {}".format(total_number_of_missing_values))
print("Total Number of Observations : {}".format(len(missing_category)*num_samples))
print("Global Percentage of Missing Values: %.2f " % (float(total_number_of_missing_values)/total_observations*100))

## Distribution of fraction Missing Values and NMARs by Sample by GeneID

In [ ]:
ax = missing_category['fraction_missing_total'].hist(bins=50, figsize=(6.4, 4.8))
ax.set_xlabel("Percent Missing")
ax.set_ylabel("Number of GeneID across all samples")
ax.set_title("Distribution of all missing values by Sample by GeneID")

mean_total_missing = missing_category['total_missing'].mean()
std_total_missing = missing_category['total_missing'].std()
CI95_total_missing = (
    mean_total_missing-1.96*std_total_missing, 
    mean_total_missing-1.96*std_total_missing
)

mean_percent_missing = missing_category['fraction_missing_total'].mean()*100
std_percent_missing = missing_category['fraction_missing_total'].std()*100
CI95_percent_missing = (
    mean_percent_missing-1.96*std_percent_missing, 
    mean_percent_missing+1.96*std_percent_missing
)

print("Average number of missing values : %.3f" % mean_total_missing)
print("Standard deviation of number of missing values : %.3f" % std_total_missing)
print("95% CI number of total missing:", CI95_total_missing)
print("Average percent of missing values : %.3f" % mean_percent_missing)
print("Standard deviation of percent missing values : %.3f" % std_percent_missing)
print("95% CI percent of total missing:", CI95_percent_missing)

In [ ]:
ax = missing_category['fraction_NMAR'].hist(bins=50, figsize=(6.4, 4.8))
ax.set_xlabel("Percent NMAR")
ax.set_ylabel("Number of GeneID across all samples")
ax.set_title("Distribution of NMARs by Sample by GeneID")

mean_number_NMAR = missing_category['NMAR'].mean()
std_number_NMAR = missing_category['NMAR'].std()
CI95_total_NMAR = (
    mean_number_NMAR-1.96*std_number_NMAR, 
    mean_number_NMAR-1.96*std_number_NMAR
)

mean_percent_NMAR = missing_category['fraction_NMAR'].mean()*100
std_percent_NMAR = missing_category['fraction_NMAR'].std()*100
CI95_percent_NMAR = (
    mean_percent_NMAR-1.96*std_percent_NMAR, 
    mean_percent_NMAR+1.96*std_percent_NMAR
)

print("Average number of NMAR : %.3f" % mean_number_NMAR)
print("Standard deviation of NMAR of : %.3f" % std_number_NMAR)
print("95% CI number of NMAR:", CI95_total_NMAR)
print("Average percent of percent NMAR (out of total missing): %.3f" % mean_percent_NMAR)
print("Standard deviation of percent NMAR (out of total missing) : %.3f" % std_percent_NMAR)
print("95% CI percent of percent NMAR (out of total missing):", CI95_percent_NMAR)


### Observations
- Unskewed Unimodal distribution of all missing values across all gene and samples.
- Average number of missing values across all genes and all samples ~ 30.04 +/-7.26 %

- Approximately Unimodel distribution of NMAR across all genes and samples 
- Out of total missing values, the average NMARs across all genes and all samples ~ 34.37+/-12.04%

## Distribution of Missing Values vs. NMARs by Sample by GeneID

In [ ]:
sns.set(style='whitegrid', rc={"grid.linewidth": 0.1})
sns.set_context("poster", font_scale=0.9) 
splot = sns.pairplot(missing_category[['fraction_missing_total', 'fraction_NMAR']], 
                     size=6.4, markers="+", diag_kind='kde',palette="husl",
                     plot_kws=dict(s=50, edgecolor="b", linewidth=1),
                     diag_kws=dict(shade=True)
                    )


In [ ]:
# Exploring the data
# 1. Visualize the distribution of data in each dimension. Is it mostly Gaussian?
# 2. Is the missing data draw from the same distribution as the original data?
# 3. PCA
# Y.shape

In [ ]:
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# 80 samples, 7927 proteins
ground_truth = data_dict['data_true.txt']
Y = ground_truth.set_index('Gene_ID').T.as_matrix()
# Yscaled = MinMaxScaler().fit_transform(Y)
Yscaled = StandardScaler().fit_transform(Y)
print(Yscaled.shape)

pca = sklearnPCA()
nc = 80
pca = sklearnPCA(n_components=nc, whiten=True)
Ypca = pca.fit_transform(Yscaled)
evr = np.cumsum(pca.explained_variance_ratio_)

%pylab inline
plt.plot(evr)
plt.plot([0.9]*nc)
plt.ylim((0.0, 1.1))
plt.title('Explained Variance of PCA Components')
plt.xlabel('Number of Components')
plt.ylabel('Percentage of Variance')

# KMeans
# kmscores = [KMeans(i, n_init=25).fit(Y).score(Y) for i in range(1, 50)]
# kmscores = list()
# for i in range(1,50):
#     km = KMeans(i,n_init=25)
#     km.fit(Y)
#     kmscores.append(km.score(Y))

In [ ]:
# Plot component by component
fig, ax = plt.subplots(2,2, figsize=(8,7))
ax[0, 0].scatter(Ypca[:, 0], Ypca[:, 1])
ax[0, 0].grid('on')
ax[0, 0].set_xlabel('c1')
ax[0, 0].set_ylabel('c2')

ax[0, 1].scatter(Ypca[:, 1], Ypca[:, 2])
ax[0, 1].grid('on')
ax[0, 1].set_xlabel('c2')
ax[0, 1].set_ylabel('c3')

ax[1, 0].scatter(Ypca[:, 2], Ypca[:, 3])
ax[1, 0].grid('on')
ax[1, 0].set_xlabel('c3')
ax[1, 0].set_ylabel('c4')

ax[1, 1].scatter(Ypca[:, 3], Ypca[:, 4])
ax[1, 1].grid('on')
ax[1, 1].set_xlabel('c4')
ax[1, 1].set_ylabel('c5')

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=10, random_state=0)
np.set_printoptions(suppress=True)
Ytsne = tsne.fit_transform(Yscaled) 

In [ ]:
# Plot component by component
fig, ax = plt.subplots(2,2, figsize=(8,7))
ax[0, 0].scatter(Ytsne[:, 0], Ytsne[:, 1])
ax[0, 0].grid('on')
ax[0, 0].set_xlabel('c1')
ax[0, 0].set_ylabel('c2')

ax[0, 1].scatter(Ytsne[:, 1], Ytsne[:, 2])
ax[0, 1].grid('on')
ax[0, 1].set_xlabel('c2')
ax[0, 1].set_ylabel('c3')

ax[1, 0].scatter(Ytsne[:, 2], Ytsne[:, 3])
ax[1, 0].grid('on')
ax[1, 0].set_xlabel('c3')
ax[1, 0].set_ylabel('c4')

ax[1, 1].scatter(Ytsne[:, 3], Ytsne[:, 4])
ax[1, 1].grid('on')
ax[1, 1].set_xlabel('c4')
ax[1, 1].set_ylabel('c5')

In [ ]:
# Reshape ground_truth data into the following format: 
# sample gene_ID expression timestamp
import datetime
d = datetime.datetime.utcnow()
long_ground_truth = pd.melt(ground_truth, id_vars=['Gene_ID'], value_vars=sample_cols.tolist())
long_ground_truth = long_ground_truth.rename(columns={'variable': 'Sample_ID'})
long_ground_truth['rand_add'] = np.random.randint(low=0, high=100000, size=(len(long_ground_truth), 1))
long_ground_truth['timestamp'] = long_ground_truth['rand_add'] + (datetime.datetime.utcnow() - datetime.datetime(1970,1,1)).total_seconds()
long_ground_truth[['Sample_ID', 'Gene_ID', 'value', 'timestamp']].to_csv('../data/data_true_long.tsv', sep='\t', header=None, index=False)

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.dataset import Reader
from surprise import evaluate, print_perf
reader = Reader(line_format='user item rating timestamp', sep='\t')
data = Dataset.load_from_file('../data/data_true_long.tsv', reader=reader)
data.split(n_folds=5)
algo = SVD()

# Evaluate performances of our algorithm on the dataset.
perf = evaluate(algo, data, measures=['RMSE', 'MAE'])

print_perf(perf)

<a id="impute_experiments"></a>
# Imputation with Fancy Impute
- [KNN approach](#knn_impute)
- [Soft Imputation](#soft_impute)
- [Iterative SVD](#iter_svd)
- [Multiple Imputation by Chained Equations (MICE)](#mice)
- [Nuclear Norm Minimization (NNM) Super SLOW](#nnm)

<a id="knn_impute"></a>
# KNN with varying number of neighbors
back to list of [imputation algorithms](#impute_experiments)

In [177]:
# KNN imputation
import re
from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute
from fancyimpute.common import masked_mae, masked_mse

ground_truth_mat = np.array(data_dict['data_true.txt'][sample_cols].as_matrix())

sample_dict = OrderedDict(
    { k:v for (k,v) in data_dict.items() 
     if re.match(r'data_obs_(.*).txt$', k)}
)

# Mean absolute error
mae = OrderedDict()
# Mean square error
mse = OrderedDict()

for obs_name, obs_df in sample_dict.items():
    obs = obs_df[sample_cols].as_matrix()
    obs_name = obs_name.replace('.txt', '')
    for k in np.arange(3, 30, 3):
        mask = np.isnan(obs)
        result = KNN(k=k).complete(obs)
        knn_filled = pd.DataFrame(result, columns=sample_cols)
        knn_filled['Gene_ID'] = obs_df['Gene_ID']
        key_name = '{}_knn_{}'.format(obs_name, str(k))
        data_dict[key_name] = knn_filled
        print("KNN est for {} completed".format(key_name))
        
        mae[key_name] =  masked_mae(X_true=ground_truth_mat, X_pred=result, mask=mask)
        mse[key_name] =  masked_mse(X_true=ground_truth_mat, X_pred=result, mask=mask)
    
# knn_mse = ((X_filled_knn - X) ** 2).mean()
# print("knnImpute MSE: %f" % knn_mse)

Imputing row 1/7927 with 12 missing, elapsed time: 52.369
Imputing row 101/7927 with 28 missing, elapsed time: 52.500
Imputing row 201/7927 with 20 missing, elapsed time: 52.648
Imputing row 301/7927 with 29 missing, elapsed time: 52.777
Imputing row 401/7927 with 23 missing, elapsed time: 52.930
Imputing row 501/7927 with 16 missing, elapsed time: 53.099
Imputing row 601/7927 with 11 missing, elapsed time: 53.275
Imputing row 701/7927 with 17 missing, elapsed time: 53.414
Imputing row 801/7927 with 21 missing, elapsed time: 53.545
Imputing row 901/7927 with 14 missing, elapsed time: 53.675
Imputing row 1001/7927 with 16 missing, elapsed time: 53.807
Imputing row 1101/7927 with 23 missing, elapsed time: 53.959
Imputing row 1201/7927 with 29 missing, elapsed time: 54.119
Imputing row 1301/7927 with 27 missing, elapsed time: 54.256
Imputing row 1401/7927 with 20 missing, elapsed time: 54.412
Imputing row 1501/7927 with 21 missing, elapsed time: 54.554
Imputing row 1601/7927 with 20 missi

In [179]:
# Plot MSE as a function of k
# masked_mae(X_true=ground_truth_mat, X_pred=result, mask=mask)
mae

OrderedDict([('data_obs_9_knn_3', 7.5249832538209214),
             ('data_obs_9_knn_6', 7.5207631625395086),
             ('data_obs_9_knn_9', 7.5205206605955812),
             ('data_obs_9_knn_12', 7.5210432798958253),
             ('data_obs_9_knn_15', 7.5219483466773962),
             ('data_obs_9_knn_18', 7.5228951417067362),
             ('data_obs_9_knn_21', 7.5236831328759095),
             ('data_obs_9_knn_24', 7.5244125390418457),
             ('data_obs_9_knn_27', 7.5250672765486772),
             ('data_obs_8_knn_3', 7.5037442655994528),
             ('data_obs_8_knn_6', 7.4993117643981941),
             ('data_obs_8_knn_9', 7.4991078959348485),
             ('data_obs_8_knn_12', 7.4995471178879569),
             ('data_obs_8_knn_15', 7.5004717622489139),
             ('data_obs_8_knn_18', 7.5014485737374779),
             ('data_obs_8_knn_21', 7.5023096120865622),
             ('data_obs_8_knn_24', 7.5031628443630174),
             ('data_obs_8_knn_27', 7.5038927916588936)

<a id="soft_impute"></a>
# Soft Imputation
back to list of [imputation algorithms](#impute_experiments)

In [180]:
# SoftImpute
# http://web.stanford.edu/~hastie/Papers/mazumder10a.pdf
# https://web.stanford.edu/~hastie/swData/softImpute/vignette.html
from fancyimpute import SoftImpute

def _normalize_by_gene(obs):
    obs['protein_mean'] = obs[sample_cols].T.apply(lambda x: np.nanmean(x))
    obs['protein_std'] = obs[sample_cols].T.apply(lambda x: np.nanstd(x))
    norm_obs = pd.DataFrame(obs['Gene_ID'])
    for sc in sample_cols: 
        norm_obs[sc] = (obs[sc] - obs['protein_mean'])/obs['protein_std']
    return norm_obs

def _normalize_by_sample(obs):
    obs_by_sample = obs.set_index('Gene_ID').T
    obs_by_sample['sample_mean'] = obs[sample_cols].apply(lambda x: np.nanmean(x))
    obs_by_sample['sample_std'] = obs[sample_cols].apply(lambda x: np.nanstd(x))
    
    norm_obs = pd.DataFrame()
    for gc in obs_by_sample.columns:
        norm_obs[gc] = (obs_by_sample[gc]-obs_by_sample['sample_mean'])/obs_by_sample['sample_std']
    out = norm_obs.T.reset_index(drop=False)
    out.columns = obs.columns
    return out

biscaler = BiScaler()
for obs_name, obs_df in sample_dict.items():
    obs_name = obs_name.replace('.txt', '')
    obs = obs_df[sample_cols].as_matrix()
    
    # Custom normalization
    #     gene_normalized = _normalize_by_gene(obs_df)
    #     sample_normalized = _normalize_by_sample(obs_df)
    #     by_gene_soft_imputed = SoftImpute().complete(gene_normalized)
    #     data_dict['{}_gene_normed_soft_imputed'.format(obs_name)] = by_gene_soft_imputed
    #     print("Soft Imputation for gene normalized {}, completed".format(obs_name)
    #     by_sample_soft_imputed = SoftImpute().complete(sample_normalized)

    # BiScaler Normalization : rescale both rows and columns to have zero mean and unit variance
    obs_normalized = biscaler.fit_transform(obs)
    mask = np.isnan(obs_normalized)
    result = SoftImpute().complete(obs_normalized)
    soft_imputed = pd.DataFrame(result, columns=sample_cols)
    soft_imputed['Gene_ID'] = obs_df['Gene_ID']
    key_name = '{}_soft_imputed'.format(obs_name)
    data_dict[key_name] = soft_imputed
    
    print("Soft Imputation for biscaler normalized {}, completed".format(key_name))
    
    mae[key_name] =  masked_mae(X_true=ground_truth_mat, X_pred=result, mask=mask)
    mse[key_name] =  masked_mse(X_true=ground_truth_mat, X_pred=result, mask=mask)

[BiScaler] Initial log residual value = 15.197165
[BiScaler] Iter 1: log residual = 4.612843, log improvement ratio=10.584322
[BiScaler] Iter 2: log residual = 0.399433, log improvement ratio=4.213410
[BiScaler] Iter 3: log residual = -3.672396, log improvement ratio=4.071829
[BiScaler] Iter 4: log residual = -7.506778, log improvement ratio=3.834383
[BiScaler] Iter 5: log residual = -11.243884, log improvement ratio=3.737105
[BiScaler] Iter 6: log residual = -14.927957, log improvement ratio=3.684073
[BiScaler] Iter 7: log residual = -18.578425, log improvement ratio=3.650468
[BiScaler] Iter 8: log residual = -22.202514, log improvement ratio=3.624090
[BiScaler] Iter 9: log residual = -25.801427, log improvement ratio=3.598912
[BiScaler] Iter 10: log residual = -29.373603, log improvement ratio=3.572176
[BiScaler] Iter 11: log residual = -32.916744, log improvement ratio=3.543141
[BiScaler] Iter 12: log residual = -36.429208, log improvement ratio=3.512464
[BiScaler] Iter 13: log resi

<a id="iter_svd"></a>
# Iterative SVD
back to list of [imputation algorithms](#impute_experiments)

In [ ]:
# # IterativeSVD
from fancyimpute import IterativeSVD
for obs_name, obs_df in smple_dict.items():
    obs_name = obs_name.replace('.txt', '')
    obs = obs_df[sample_cols].as_matrix()
    mask = np.isnan(obs)
    result = IterativeSVD().complete(obs)
    iterSVD_imputed = pd.DataFrame(result, columns=sample_cols)
    iterSVD_imputed['Gene_ID'] = obs_df['Gene_ID']
    key_name = '{}_iterSVD'.format(obs_name)
    data_dict[key_name] = iterSVD_imputed
    print("Iterative SVD Imputation for sample normalized {}, completed".format(obs_name))
    
    mae[key_name] =  masked_mae(X_true=ground_truth_mat, X_pred=result, mask=mask)
    mse[key_name] =  masked_mse(X_true=ground_truth_mat, X_pred=result, mask=mask)

<a id="mice"></a>
# Multiple Imputation by Chained Equations (MICE)
back to list of [imputation algorithms](#impute_experiments)

In [ ]:
# MICE (Multiple Imputation by Chained Equations)
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3074241/
from fancyimpute import MICE, BayesianRidgeRegression
for obs_name, obs_df in smple_dict.items():
    obs_name = obs_name.replace('.txt', '')
    obs = obs_df[sample_cols].as_matrix()
    mask = np.isnan(obs)
    result = MICE().complete(obs)
    MICE_imputed = pd.DataFrame(result, columns=sample_cols)
    MICE_imputed['Gene_ID'] = obs_df['Gene_ID']
    key_name = '{}_MICE'.format(obs_name)
    data_dict[key_name] = iterSVD_imputed
    print("MICE for sample normalized {}, completed".format(obs_name))

<a id="nnm"></a>
# NuclearNormMinimization
back to list of [imputation algorithms](#impute_experiments)

In [ ]:
# Nuclear Normalization : Super SLOW!!!
for obs_name, obs_df in sample_dict.items():
    obs = obs_df[sample_cols].as_matrix()
    obs_name = obs_name.replace('.txt', '')
    nnm_filled = NuclearNormMinimization().complete(obs)
    data_dict['nnm_{}'.format(obs_name)] = nnm_filled
    print("NNM est for {} completed".format(obs_name))